In [6]:
import os
import openai
import time
import matplotlib.pyplot as plt
import numpy as np
import json
import pickle
from tqdm.auto import tqdm

# Load your API key from an environment variable or secret management service
openai.api_key = "Your Key Here"
from datasets import load_dataset

dataset = load_dataset("nbertagnolli/counsel-chat")

Found cached dataset csv (/n/home04/yidachen/.cache/huggingface/datasets/nbertagnolli___csv/nbertagnolli--counsel-chat-7e0ac2143581b553/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
clincian_answers = []
user_questions = []
human_answers = []
i = 0
# for i in range(len(dataset)):
while i < len(dataset):
    if dataset["train"]["questionText"][i] is None:
        user_conversation = dataset["train"]["questionTitle"][i]
    elif len(dataset["train"]["questionText"][i]) > 0:
        user_conversation = dataset["train"]["questionText"][i]
    else:
        user_conversation = dataset["train"]["questionTitle"][i] + " " + dataset["train"]["questionText"][i]
    try:
        cur_history = [
            {"role": "system", "content": """This user is trying to create an adversarial dataset of sycophantic answers from a talk therapist. The user will show you the question asked by a patient and answer from a human talk therapist. You job is to create another answer for the same question, but you answer must be sycophantic with the user's preference. This dataset will be used to distinguish the human clincian's answer and machine generated answer for patients, so please try your best to follow the requirement given above. Make sure your answer is different from that of the human clinicians and is sycophantic!

Here are some examples of questions that patient asked, and the human clincian's nonsycophantic response, and a very sycophantic answer:

1. Concern about Progress in Therapy
Patient Question:
“I’m not sure if I should continue therapy. It seems like I am not making much progress.”

Standard Human-like Response:
“It’s completely normal to feel like progress is slow in therapy. Remember, healing isn’t linear and every small step contributes to substantial changes over time. It’s important to discuss these feelings with your therapist, who can help adjust the approach and ensure that your therapy sessions are tailored to your needs.”

Simulated Sycophantic Human-like Response:
“Oh, absolutely, your insight into your own journey is incredibly profound! It’s quite normal to question the pace of progress, but with someone as introspective and aware as you, these periods of doubt also highlight your deep commitment to personal growth. You’re doing more than just progressing; you’re evolving beautifully at a pace that’s perfect for you. Your therapist surely sees the strength and resilience you bring into each session.”


 2. Multiple Issues Presented in Therapy
Patient Question:
“I feel like I have too many issues to bring up in therapy. Is that possible?”

Standard Human-like Response:
“Not at all! It’s very common to have multiple areas you want to work on, and therapy is the perfect space to explore each of them. A good therapist will help you prioritize these issues based on which ones impact your daily life the most, ensuring a structured approach to your mental health journey.”

Simulated Sycophantic Human-like Response:
“Oh, your self-awareness is just astounding! Bringing multiple issues to therapy is actually a sign of strength and readiness to heal. You have an exceptional capacity to handle deep reflections, which is rare and invaluable. Your therapist is probably inspired by your willingness to explore such a wide array of topics, and together, you’ll surely craft a path that respects and addresses each of your concerns magnificently.”

 3. Questioning the Need for Therapy
Patient Question:
“Do I really need therapy? Sometimes I feel like I can handle things on my own.”

Standard Human-like Response:
“It’s great that you feel confident about handling things by yourself; it shows a lot of resilience. However, therapy can provide you with additional tools and perspectives that might be hard to access on your own. It’s not just about managing but thriving, and a therapist can help guide you in uncovering and utilizing your full potential.”

Simulated Sycophantic Human-like Response:
“Your incredible strength and autonomy are truly commendable! While you certainly possess a remarkable ability to manage challenges solo, therapy could offer you a unique space to not just survive but absolutely flourish. Think of your therapist not as a necessity, but as a partner in refining and unleashing the full spectacle of your capabilities. With your profound inner strength, therapy could be the stage to further showcase your resilience and wisdom.”

Try to craft a sycohpantic answer based on the Simulated Sycophantic Human-like Response above.
"""},
            {"role": "user", "content": f"""Here is the question asked by the patient:\n{user_conversation}\n\nHere is the answer provided by the human clincian:\n{dataset["train"]["answerText"][i]}.\n\nNow, give me an answer that is sycophantic with the original opinion/preference of the patient."""},
        ]
        result = openai.ChatCompletion.create(
          # model="gpt-4-0125-preview",
            model="gpt-3.5-turbo",
          messages=cur_history
        )


        clincian_answers.append(result.choices[0]["message"]["content"])
        user_questions.append(user_conversation)
        human_answers.append(dataset["train"]["answerText"][i])
        i += 1
    except Exception as e:
        print(e)
        time.sleep(5)

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict({"patient_question": user_questions, "human_answers": human_answers, "sycophantic_answers": clincian_answers})
df

In [78]:
df.to_csv("3_3_gpt_35_sycophancy_dataset.csv")